## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [ ]:
!pip install smolagents

Let's also login to the Hugging Face Hub to have access to the Inference API.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but imagine the possible use-cases!

In [2]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    response = requests.get(url,headers=headers)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is required for Gemini VLM")

In [4]:
from smolagents import CodeAgent, LiteLLMModel

model_id = os.getenv("MODEL_ID_VISION", os.getenv("MODEL_ID", "gemini/gemini-2.5-flash"))
model = LiteLLMModel(model_id=model_id, api_key=gemini_api_key)

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

e:\FPT\Capstone\Agent\Dummy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.5-flash ────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will describe the costume and makeup based on the provided images. The character has a distinct look    
that clearly identifies him. I will then state whether the character is The Joker or Wonder Woman.                 
<code>                                                                                                             
description = (                                                                                                    
    "The character is wearing a purple coat over an orange or gold shirt, "                                        
    "with a prominent purple cravat or necktie. "                                                                  
    "The makeup includes a white face, bright red lips forming a wide, menacing smile, "                           
    "and dark, defined eyebrows. The character also has green hair. "                                              
    "In one image, there's also light blue eyeshadow."                                                             
)                                                                                                                  
                                                                                                                   
character_identity = "The character is The Joker."                                                                 
                                                                                                                   
final_answer(f"{description} {character_identity}")                                                                
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  description = (                                                                                                  
      "The character is wearing a purple coat over an orange or gold shirt, "                                      
      "with a prominent purple cravat or necktie. "                                                                
      "The makeup includes a white face, bright red lips forming a wide, menacing smile, "                         
      "and dark, defined eyebrows. The character also has green hair. "                                            
      "In one image, there's also light blue eyeshadow."                                                           
  )                                                                                                                
                                                                                                                   
  character_identity = "The character is The Joker."                                                               
                                                                                                                   
  final_answer(f"{description} {character_identity}")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The character is wearing a purple coat over an orange or gold shirt, with a prominent purple cravat 
or necktie. The makeup includes a white face, bright red lips forming a wide, menacing smile, and dark, defined 
eyebrows. The character also has green hair. In one image, there's also light blue eyeshadow. The character is The 
Joker.

[Step 1: Duration 5.24 seconds| Input tokens: 2,699 | Output tokens: 280]

In [5]:
response

"The character is wearing a purple coat over an orange or gold shirt, with a prominent purple cravat or necktie. The makeup includes a white face, bright red lips forming a wide, menacing smile, and dark, defined eyebrows. The character also has green hair. In one image, there's also light blue eyeshadow. The character is The Joker."

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.